<a href="https://colab.research.google.com/github/Sunn24/Fermat-Factorization/blob/main/Tigtagtoe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ทดลองหา Winning rate ของ Eggy chess (จาก RO)
## code 80% ช่วยเขียนโดย ChatGPT ดังนั้น ถ้ามีอะไรผิดพลาด... ไปด่า ChatGPT 555555

import package ที่ต้องใช้ และ dataclass ไว้แทน struct

In [ ]:
import random
from dataclasses import dataclass

สร้าง PlayerInfo ไว้บอกว่ากำหนดว่า มีจำนวน หมากใหญ่, กลาง, เล็ก กี่ตัว? และ เดินไปแล้วกี่ครั้ง (ที่เก็บแยกตัวแปร เพราะเผื่อใช้สำหรับ UI)

In [ ]:
@dataclass
class PlayerInfo:
    big_move_limit: int
    medium_move_limit: int
    small_move_limit: int
    big_moves_used: int
    medium_moves_used: int
    small_moves_used: int

Check ผู้ชนะ XO แบบตาราง 3x3

In [ ]:
def check_winner(board):
    # Check rows, columns, and diagonals for a winner
    for row in board:
        if all(cell == row[0] and cell != ' ' for cell in row):
            return row[0]

    for col in range(3):
        if all(board[row][col] == board[0][col] and board[row][col] != ' ' for row in range(3)):
            return board[0][col]

    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != ' ':
        return board[0][0]

    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != ' ':
        return board[0][2]

    return None

* select_movable_cell function ที่จะให้ช่องที่ player สามารถเลือกลงหมากได้ คนเป็น array 1D ที่มีค่าของ tuple x,y (index ช่องที่เลือกได้)
  *   board = ตารางปัจจุบัน ใครวางส่วนไหน เป็น array string 3x3 เป็นไปได้ 3 values ('X', 'O', ' ')
  *   move_sizes = ตารางปัจจุบัน แต่บอกเป็น size ของหมากที่วาง เป็น array int 3x3 เป็นไปได้ 4 values (3, 2, 1, 0)
    * 3 คือ หมากใหญ่สุด
    * 2 คือ หมากกลาง
    * 1 คือ หมากเล็ก
    * 0 คือ ยังไม่วาง
  * move_size คือ player กำลังจะวางหมากอะไร (3,2,1)
  * current_player คือ player คนนี้คือใคร ('X', 'O')



In [ ]:
def select_movable_cell(board, move_sizes, move_size, current_player):
    # get all cell
    all_cells = [(i, j) for i in range(3) for j in range(3) if (board[i][j] != current_player and move_sizes[i][j] < move_size) or board[i][j] == ' ']
    if not all_cells:
        return None  # Draw
    return all_cells



*   select_move_size ไว้ random ขนาดหมากของ player return เป็น 3,2,1
  * อาจจะเกิด error ได้ถ้า ไม่เข้า if ไหนเลย


In [ ]:
def select_move_size(player_info):
    choices = []
    if player_info.big_moves_used < player_info.big_move_limit:
        choices.append(3)
    if player_info.medium_moves_used < player_info.medium_move_limit:
        choices.append(2)
    if player_info.small_moves_used < player_info.small_move_limit:
        choices.append(1)
    return random.choice(choices)

* find_winner_by_count_player ไว้รับว่าหมากใครมากกว่ากันในกระดาน return เป็น 'X' กับ 'O' (ไว้เช็คคนชนะตอนท้าย)

In [ ]:
def find_winner_by_count_player(board):
    countX = 0
    countY = 0
    for player in board:
        if player == 'X':
            countX += 1
        else:
            countY += 1
    return 'X' if countX > countY else 'O'

# Simulate game


* เล่นเกมจนผ่าน
* return เป็นผู้ชนะ ('X', 'O', None)
* เป็น None ถ้าไม่มีผู้ชนะ (อันนี้ไม่ควรเกิดขึ้น เพราะ แก้เป็นนับจำนวนหมากแทน)
* แก้ PlayeInfo เพื่อ balance เกม



In [ ]:
def simulate_game():
    board = [
        [' ', ' ', ' '],
        [' ', ' ', ' '],
        [' ', ' ', ' ']
    ]

    move_sizes = [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ]

    player1 = 'X'
    player2 = 'O'
    current_player = player1

    # Change this section for tuning parameter
    # TODO - Refactor by Extract this part as parameters
    # player 1 is starter
    # ***********************************************************************************************
    # EDIT THIS PART TO BALANCE GAME
    # ***********************************************************************************************
    player1_info = PlayerInfo(big_move_limit=2, medium_move_limit=3, small_move_limit=3,
                              big_moves_used=0, medium_moves_used=0, small_moves_used=0)

    # player 2
    player2_info = PlayerInfo(big_move_limit=2, medium_move_limit=3, small_move_limit=3,
                              big_moves_used=0, medium_moves_used=0, small_moves_used=0)
    # ***********************************************************************************************

    # Maximum 9 moves on a 3x3 board, if it's normal Tig tag toe
    # but now it's 16, Because we can replace other mark with bigger one
    for k in range(16):
        # get all cells that empty
        empty_cells = [(i, j) for i in range(3) for j in range(3) if board[i][j] == ' ']

        # if no empty cells mean game ended then find winner
        if not empty_cells:
            winner = find_winner_by_count_player(board)
            return winner

        # assign player info for this turn
        current_player_info = player1_info if current_player == player1 else player2_info

        # select move based on what player can choose
        move_size = select_move_size(current_player_info)

        # if first turn select medium size
        if k == 1:
          move_size = 2

        all_cells = select_movable_cell(board, move_sizes, move_size, current_player)

        # if can't not play then random again until playable
        while not all_cells:
            move_size = move_size = select_move_size(current_player_info)
            all_cells = select_movable_cell(board, move_sizes, move_size, current_player)

        # random play
        row, col = random.choice(all_cells)

        # Handle the chosen move size
        if move_size == 3:  # Big
            # can play big move? (this condition is unnecessary because we already checked in select_movable_cell)
            # TODO - remove this but don't remove "current_player_info.big_moves_used += 1"
            if current_player_info.big_moves_used < current_player_info.big_move_limit and move_sizes[row][col] < 3:
                board[row][col] = current_player
                current_player_info.big_moves_used += 1
        elif move_size == 2:  # Medium
            # can play Medium move? (this condition is unnecessary because we already checked in select_movable_cell)
            # TODO - remove this but don't remove "current_player_info.medium_moves_used += 1"
            if current_player_info.medium_moves_used < current_player_info.medium_move_limit and move_sizes[row][col] < 2:
                board[row][col] = current_player
                move_sizes[row][col] = 2
                current_player_info.medium_moves_used += 1
        else:  # Small
            # can play Small move? (this condition is unnecessary because we already checked in select_movable_cell)
            # TODO - remove this but don't remove "current_player_info.small_moves_used += 1"
            if board[row][col] == ' ' and current_player_info.small_moves_used < current_player_info.small_move_limit:
                board[row][col] = current_player
                move_sizes[row][col] = 1
                current_player_info.small_moves_used += 1

        # check winner after play
        winner = check_winner(board)
        if winner:
            return winner

        # next turn, Change player
        current_player = player2 if current_player == player1 else player1

    # End game after play then Find winner
    return find_winner_by_count_player(board)

# calculate_win_rates_simulated
for count winner


*   num_simulations คือ จำนวนเกมที่จะเล่น
*   ยิ่งเล่นหลายเกมยิ่งแม่น
*   ลองเล่นแบบ XO ปกติแล้วตรงกับที่มีคนคำนวนไว้พอดี http://people.missouristate.edu/lesreid/sol10_04.html



In [ ]:
def calculate_win_rates_simulated(num_simulations):
    player1_wins = 0
    player2_wins = 0
    draws = 0

    for _ in range(num_simulations):
        result = simulate_game()
        if result == 'X':
            player1_wins += 1
        elif result == 'O':
            player2_wins += 1
        else:
            draws += 1

    player1_win_rate = player1_wins / num_simulations
    player2_win_rate = player2_wins / num_simulations
    draw_rate = draws / num_simulations

    return player1_win_rate, player2_win_rate, draw_rate

In [ ]:
num_simulations = 100000  # Adjust the number of simulations as needed
player1_win_rate, player2_win_rate, draw_rate = calculate_win_rates_simulated(num_simulations)
print("Player 1 Win Rate:", player1_win_rate)
print("Player 2 Win Rate:", player2_win_rate)
print("Draw Rate:", draw_rate)

Player 1 Win Rate: 0.56149
Player 2 Win Rate: 0.43851
Draw Rate: 0.0


# In conclusion


*   ยังไม่มีทางที่จะได้ Fair game (50:50 winning rate)
  * แต่สามารถหา win rate ที่ใกล้เคียง 50 ได้
  * โดยคนแรกจะมีโอกาสชนะมากกว่า ประมาณ 5% (ไม่มีทางเสมอ)
  * เมื่อเทียบกับ Tig tag toe ปกติที่ต่างกันถึง 18%(รวมเสมอ) - 30%(ไม่รวมเสมอ)
*   สิ่งที่ได้ลองไปแล้ว
  *  พบว่าหมากใหญ่สุดคือ advantage ที่สำคัญสุด
  *  Ratio ที่ใกล้เคียง 50% ที่สุดคือ
    * คนแรกได้ หมากใหญ่ = 1, หมากกลาง = 3, หมากเล็ก = 4 (ย้ายอันใหญ่มาเป็นอันเล็ก)
    * คนสองได้ หมากใหญ่ = 2, หมากกลาง = 3, หมากเล็ก = 3 (ใหญ่ 2 เผื่อแก้เกม)
*   สิ่งที่ยังไม่ได้ลอง และ น่าจะ improve ได้
  * เพิ่มขนาดบอร์ดเป็น 5x5? (เพิ่ม ความเป็นไปได้ แต่ไม่แน่ใจว่าลดความต่าง winning rate ไหม)
  * (idk ยังคิดไม่ออก)
*   ความคิดเห็น
  * เหมาะมากที่จะเป็น mini game มากกว่าที่จะเป็น standalone (Zoom-out pivot)
  * ความเป็นไปได้ สำหรับคนเล่นเป็นแล้ว มันจะน้อยทำให้เบื่อง่าย รึเปล่า? (More features)
      * อาจจะต้องมีตัวแปรอื่นเพิ่ม? (เพิ่มบางอย่างเข้าไป)

# Minimax Implementation

In [ ]:
import sys
import random

In [ ]:
# The TicTacToe board is represented as a list of lists (3x3 grid)
# Empty cells are represented as ' ', X and O represent the players
board = [[' ' for _ in range(3)] for _ in range(3)]
board_size = [[0 for _ in range(3)] for _ in range(3)]
X_sizes = [3, 3, 2]
O_sizes = [3, 3, 2]

transposition_table = {}

In [ ]:
# Function to print the current state of the board
def print_board(board, board_size):
    for row in board:
        print('|'.join(row))
        print('-' * 5)
    for row in board_size:
        print('|'.join(str(row)))
        print('-' * 5)

In [ ]:
def get_possible_size(current_sizes, validate):
    size_availables = []
    for i in range(3):
      if current_sizes[i] > 0:
          size_availables.append(i + 1)

    if validate:
        return size_availables

    # ************ Write you rule to pick size here **************
    # enable 2 size for find more win possibilities
    # if  sum(X_sizes) < 6:
    #     return size_availables[-2:]

    if sum(X_sizes) > 6:
        return [size_availables[1]]

    if  sum(X_sizes) < 5:
        return size_availables

    # default return only 1 size for fast finding
    if len(size_availables) > 0:
        return size_availables[-2:]
    return size_availables

In [ ]:
def find_choices(board, board_size, side, current_sizes, validate=False):
    available_choices = []
    possible_size = get_possible_size(current_sizes, validate)
    for p_size in possible_size:
        for row in range(3):
            for col in range(3):
                isOwnMark = board[row][col] == side
                canPlace = board_size[row][col] < p_size
                sizeBlock = (p_size - 1) * 9
                rowBlock = row * 3
                if not isOwnMark and canPlace:
                    available_choices.append(sizeBlock + rowBlock + col)
    return available_choices

In [ ]:
# Function to check if the board is full
def is_board_full(side, current_sizes):
    choices = find_choices(board, board_size, side, current_sizes, validate=True)
    can_place_somewhere = len(choices) > 0

    # is can't replace any mark
    return not can_place_somewhere
    # return all(cell != ' ' for row in board for cell in row)

In [ ]:
# Function to check if a player has won
def is_winner(player):
    # Check rows, columns, and diagonals
    for i in range(3):
        if all(board[i][j] == player for j in range(3)) or \
           all(board[j][i] == player for j in range(3)):
            return True
    if all(board[i][i] == player for i in range(3)) or \
       all(board[i][2 - i] == player for i in range(3)):
        return True
    return False

In [ ]:
def check_draw_win(player):
    target = 0
    for i in range(3):
        for j in range(3):
            if board[i][j] == player:
                target += 1
    return target > 4

In [ ]:
def get_hash_board():
    return tuple([tuple(tuple(row) for row in board), tuple(tuple(row) for row in board_size)])

In [ ]:
def minimax(depth, is_maximizing, alpha, beta):

    # hash_board = get_hash_board()
    # if hash_board in transposition_table:
    #     return transposition_table[hash_board]

    if is_winner('X'):
        return -1
    if is_winner('O'):
        return 1
    if is_board_full('O' if is_maximizing else 'X', O_sizes if is_maximizing else X_sizes):
        return 0.5 if check_draw_win('O') else -0.5

    if is_maximizing:
        max_eval = -sys.maxsize
        possible_choices = find_choices(board, board_size, 'O', O_sizes)
        for p_choice in possible_choices:
            select_size = int(p_choice / 9 + 1)
            slot = p_choice % 9
            row = int(slot / 3)
            col = slot % 3

            prev_size = board_size[row][col]
            prev_side = board[row][col]

            board[row][col] = 'O'
            board_size[row][col] = select_size
            O_sizes[select_size - 1] -= 1
            eval = minimax(depth + 1, False, alpha, beta)

            board[row][col] = prev_side
            board_size[row][col] = prev_size
            O_sizes[select_size - 1] += 1

            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break

        # transposition_table[get_hash_board()] = max_eval
        return max_eval

        # for i in range(3):
        #     for j in range(3):
        #         if board[i][j] == ' ':
        #             board[i][j] = 'O'
        #             eval = minimax(depth + 1, False)
        #             board[i][j] = ' '
        #             max_eval = max(max_eval, eval)
        # return max_eval
    else:
        min_eval = sys.maxsize
        possible_choices = find_choices(board, board_size, 'X', X_sizes)
        for p_choice in possible_choices:
            select_size = int(p_choice / 9 + 1)
            slot = p_choice % 9
            row = int(slot / 3)
            col = slot % 3

            prev_size = board_size[row][col]
            prev_side = board[row][col]

            board[row][col] = 'X'
            board_size[row][col] = select_size
            X_sizes[select_size - 1] -= 1
            eval = minimax(depth + 1, True, alpha, beta)

            board[row][col] = prev_side
            board_size[row][col] = prev_size
            X_sizes[select_size - 1] += 1

            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break

        # transposition_table[get_hash_board()] = min_eval
        return min_eval

In [ ]:
# Function to find the best move using the Minimax algorithm
def find_best_move():
    best_move = None
    best_eval = -sys.maxsize
    alpha = -sys.maxsize
    beta = sys.maxsize
    possible_choices = find_choices(board, board_size, 'O', O_sizes)
    print(O_sizes)
    print(possible_choices)
    eval_best_chocies = []
    more_best_choices = []
    for p_choice in possible_choices:
        select_size = int(p_choice / 9 + 1)
        slot = p_choice % 9
        row = int(slot / 3)
        col = slot % 3

        prev_size = board_size[row][col]
        prev_side = board[row][col]

        board[row][col] = 'O'
        board_size[row][col] = select_size
        O_sizes[select_size - 1] -= 1
        eval = minimax(0, False, alpha, beta)
        eval_best_chocies.append(eval)
        print(eval)

        # if is choice make computer win so pick it... NOW!!!
        is_instance_win = is_winner('O')

        board[row][col] = prev_side
        board_size[row][col] = prev_size
        O_sizes[select_size - 1] += 1
        more_best_choices.append((row, col, select_size))

        if is_instance_win:
            best_move = (row, col, select_size)
            return best_move

        if eval > best_eval:
            best_eval = eval
            best_move = (row, col, select_size)
            # if best_eval == 1:
            #     break

    filtered_best_choices = []
    for i in range(len(eval_best_chocies)):
        if best_eval == eval_best_chocies[i]:
            filtered_best_choices.append(more_best_choices[i])

    if len(filtered_best_choices) > 1:
        return random.choice(filtered_best_choices)

    return best_move

In [ ]:
def play_game():
    while not is_board_full('O', O_sizes) and not is_winner('X') and not is_winner('O'):
        print_board(board, board_size)
        player_row, player_col, size = map(int, input("Enter your move (row and column and size): ").split())

        isOwnMark = board[player_row][player_col] == "X"
        canPlace = board_size[player_row][player_col] < size

        if not isOwnMark and canPlace and X_sizes[size - 1] > 0:
            board[player_row][player_col] = 'X'
            board_size[player_row][player_col] = size
            X_sizes[size - 1] -= 1
        else:
            print("Invalid move. Try again.")
            continue

        if is_board_full('X', X_sizes):
            print_board(board, board_size)
            print("It's a draw!")
            isWin = check_draw_win('X')
            if isWin:
                print("You win!")
            else:
                print("Computer wins!")
            break

        if is_winner('X'):
            print_board(board, board_size)
            print("You win!")
            break

        if is_board_full('O', O_sizes):
            print_board(board, board_size)
            print("It's a draw!")
            isWin = check_draw_win('X')
            if isWin:
                print("You win!")
            else:
                print("Computer wins!")
            break

        print("Computer's turn...")
        computer_row, computer_col, computer_size = find_best_move()
        board[computer_row][computer_col] = 'O'
        board_size[computer_row][computer_col] = computer_size
        O_sizes[computer_size - 1] -= 1

        if is_winner('O'):
            print_board(board, board_size)
            print("Computer wins!")
            break

    if is_board_full('X', X_sizes):
        print("OSize:")
        print(O_sizes)
        print("----")
        print_board(board, board_size)
        print("It's a draw!")
        isWin = check_draw_win('X')
        if isWin:
            print("You win!")
        else:
            print("Computer wins!")

In [ ]:
# Start the game
play_game()

 | | 
-----
 | | 
-----
 | | 
-----
[|0|,| |0|,| |0|]
-----
[|0|,| |0|,| |0|]
-----
[|0|,| |0|,| |0|]
-----
Enter your move (row and column and size): 0 0 3
Computer's turn...
[3, 3, 2]
[10, 11, 12, 13, 14, 15, 16, 17]
-1
-1
-1
-0.5
1
-1
1
1
X| | 
-----
 | | 
-----
 |O| 
-----
[|3|,| |0|,| |0|]
-----
[|0|,| |0|,| |0|]
-----
[|0|,| |2|,| |0|]
-----
Enter your move (row and column and size): 1 1 3
Computer's turn...
[3, 2, 2]
[10, 11, 12, 14, 15, 17, 19, 20, 21, 23, 24, 26]
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
1
X| | 
-----
 |X| 
-----
 |O|O
-----
[|3|,| |0|,| |0|]
-----
[|0|,| |3|,| |0|]
-----
[|0|,| |2|,| |2|]
-----
Enter your move (row and column and size): 2 0 2
Computer's turn...
[3, 1, 2]
[10, 11, 12, 14, 19, 20, 21, 23, 24]
-1
-1
-1
-1
-1
-1
-1
-1
1
X| | 
-----
 |X| 
-----
O|O|O
-----
[|3|,| |0|,| |0|]
-----
[|0|,| |3|,| |0|]
-----
[|3|,| |2|,| |2|]
-----
Computer wins!


# Conclusion of Tic tac toe Eggy with Eggy from RO


*   เราใช้ Code เดียวกันกับ Tic tac toe ธรรมดาแต่เพิ่ม Rules เข้าไป
*   ตัวเกมมีความเป็นไปได้ที่มากกว่า Tic tac toe ปกติอย่างมาก
  * การจะสร้าง Q-Table สำหรับ Q-Learning และ Reinforcement learning ต้องใช้ resouces สูงถึง 3 GB of Ram (แต่ถ้า optimize ด้วยการ Transpose Table จะเหลือ 1 GB)
  * อย่างไรก็ตามกว่า Q-Table จะถูกเติมจนเต็มกว่า 40M ความเป็นไปได้ ก็ใช้เวลานานอย่างมาก
  * เลยไปใช้ DeepQLearning เพิ่มแก้ปัญหาจำนวน State ที่เยอะ (ถึงจะบอกว่า DQN แต่จริงๆใช้ PPO ของ OpenAI)
  * สิ่งที่ทำคือ
      * สร้าง Agent 2 ตัวแข่งกัน ปรากฏว่า ตัวเริ่มก่อนจะเก่งจนชนะเสมอ อย่างมีนัยสำคัญ ส่วนอีกตัว Throw Game (ที่บอกว่า Throw game เพราะ ไม่ว่า predit action อะไรก็แพ้ หรือ reward ติดลบ จึงเลือก random choice อยู่ดี)
      * เพื่อแก้ปัญหา Agent เก่งไม่ทันกัน เลยเอาไปแข่งกับ Random Agent เพื่อให้เกิดการเรียนรู้ กฎของเกมก่อน พบว่า AgentX (ตัวเริ่มก่อน) ได้คะแนนสูงกว่า AgentO (ตัวเริ่มหลัง) ถึง 2 เท่า แม้ model จะ Overfit แล้วก็ตาม
  * ด้วยผลแบบนี้เลยสันนิษฐานว่า
      * ความไม่ Balance ของเกมอาจมีผลต่อการเรียนรู้ของ Agent
      * ยัง Train Agent ไม่พอ
      * เกมมีความเป็นไปได้เยอะ และ Model เล็กเกินไป
      * ความอยากรู้ของ Model อาจจะน้อยเกินไปจนไม่ได้ไป พบความเป็นไปได้ที่ทำให้ชนะ
      * เกมที่แพ้อาจจะมีมากกว่าเกมที่ชนะมากเกินไป และ Batch Size ไม่พอทำให้ ติดอยู่ที่ local minima (อันนี้ไม่จริง เพิ่ม Batch Size แล้ว Overfit ไว้มากกลายเป็น เลือก choice ซ้ำๆแทน)
      * **Model ไม่เจอคู่แข่งที่คู่ควรมากพอ**
  * ในการทำ Tic tac toe Agent จะมีอีกรูปแบบนึงคือ
      * Minimax ใช้ recursive เข้าไปดูความเป็นไปได้ที่จะชนะมากที่สุด
      * แต่ด้วยความที่ความเป็นไปได้เยอะมาก จึงทำให้มันช้าสาดดดดด
          * มีบอร์ดว่าง 8 ช่อง จะใช้เวลามากกว่า 30 นาทีในการหา 1 ช่อง (นานแค่ไหนไม่รู้ เพราะ รันไม่จบ 5555)
      * เพื่อ Optimize ความเร็วเลย Fix ความเป็นไปได้ให้ใช้ 2 size ที่ขนาดใหญ่สุดก่อนเสมอ
          * ลองรันหาความเป็นไปได้แล้วว่า ลง size เล็กสุดมีแต่แพ้ ด้วยการหา minimax
          * ทำให้ speed up ได้ โดยจะสามารถหาคำตอบได้ที่ 30วิ - 1 นาทีต่อช่อง
      * เพื่อ Alpha Beta เข้าไป เพื่อกำจัดการคำนวณที่ไม่จำเป็น
          * ความเร็ว 5-10 วิ/คำตอบ
      * ใช้ Caching
          * ครั้งแรกจะนานหน่อย แต่หลังจากนั้น ใช้ไม่เกิน 1 วิ

